# Setup

In [10]:
pip install ipywidgets mlcroissant

Note: you may need to restart the kernel to use updated packages.


In [11]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Data Initialization

In [13]:
import mlcroissant as mlc
import pandas as pd

# Fetch the Croissant JSON-LD
croissant_dataset = mlc.Dataset('https://www.kaggle.com/datasets/matepapava/rotten-tomatoes-critics-and-audience-scores/croissant/download')

# Check what record sets are in the dataset
record_sets = croissant_dataset.metadata.record_sets
print(record_sets)

# Fetch the records and put them in a DataFrame
record_set_df = pd.DataFrame(croissant_dataset.records(record_set=record_sets[0].uuid))
record_set_df.head()

# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("matepapava/rotten-tomatoes-critics-and-audience-scores")

# print("Path to dataset files:", path)
# # try:
# #     df = pd.read_json(data_file)
# #     print(f"Data loaded!")
# # except Exception as e:
# #     print(f"Error encountered: {e}")

  -  [Metadata(Rotten Tomatoes: Critics & Audience scores)] Property "http://mlcommons.org/croissant/citeAs" is recommended, but does not exist.
  -  [Metadata(Rotten Tomatoes: Critics & Audience scores)] Property "https://schema.org/datePublished" is recommended, but does not exist.


[]


# Data Cleaning

In [4]:
print(df.head())

NameError: name 'df' is not defined

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]
Note: you may need to restart the kernel to use updated packages.
